In [3]:
# Import de bibliotecas
import csv, ast, psycopg2
import pandas as pd
from datetime import datetime as dt
import numpy as np


# Leitura de dados e criação de variáveis

In [4]:
# Lê dados de excel para dataframe
reader = pd.read_excel('BD.xlsx',dtype="string", engine='openpyxl')
# Confirma se todas as colunas foram importadas como string
print(reader.dtypes)

# Cria variáveis para armazenar cabeçalhos, tipos e tamanhos
longest, headers, type_list = [], [], []
# Cria variável com o nome da tabela que será criada
table_name = 'XPTO'
# Imprime amostra de dados do arquivo na tela
reader.head()

NF                     string
ID_PRODUTO             string
QTD                    string
VALOR                  string
DESCONTO_PERCENTUAL    string
DATA                   string
ID_CLIENTE             string
dtype: object


,NF,ID_PRODUTO,QTD,VALOR,DESCONTO_PERCENTUAL,DATA,ID_CLIENTE
0,01,01,9,4.4,2,2022-01-14 00:00:00,01
1,02,01,2,4.5,3,2022-01-22 00:00:00,03
2,03,01,3,4.4,1,2022-01-14 00:00:00,03
3,04,02,6,64.12,5,2022-01-21 00:00:00,03
4,05,02,5,64.1,0,2022-01-14 00:00:00,03


# Método para definição dos tipos e tamanhos das variáveis do arquivo

In [5]:
# Define função para análise dos tipos das colunas
# val representa o valor do campo e current_type o tipo do dado até o presente momento da análise
def dataType(val, current_type):
    # A primeira checagem consiste na verificação do tipo data (O algoritmo criado só trabalha com data completa)
    try:
        format = "%Y-%m-%d %H:%M:%S"
        date = dt.strptime(val, format)
        return 'timestamp'
    except:
        try:
            # A segunda checagem avalia números para o tipo apropriado
            t = ast.literal_eval(val)
        # Um erro na segunda checagem atribui o tipo varchar    
        except ValueError:
            return 'varchar'
        except SyntaxError:
            return 'varchar'
        # Caso a função não seja retornada por um try-catch
        # Se o tipo estiver entre int, long ou float    
        if type(t) in [int, 'long', float]:
            # Caso o tipo seja long ou int
            if (type(t) in [int, 'long']) and current_type not in ['float', 'varchar']:
                # Caso o tamanho da variável do tipo string for maior do que o tamanho da mesma variável convertida para inteiro
                # atribui o tipo varchar, como identificadores de variáveis e CPFs iniciados com 0
                if len(val[0]) > len(str(int(val))):
                    return 'varchar'
                # Caso contrário, atribui o menor valor possível como tipo (smallint, int ou bigint)
                elif (-32768 < t < 32767) and current_type not in ['int', 'bigint']:
                    return 'smallint'
                elif (-2147483648 < t < 2147483647) and current_type not in ['bigint']:
                    return 'int'
                else:
                    return 'bigint'
            # Valida o tamanho do varchar
            if type(t) is float and current_type not in ['varchar']:
                return 'decimal'
        # Se não estiver entre int, long ou float atribui varchar            
        else:
            return 'varchar'

# Avaliação linha a linha dos tipos de colunas do arquivo

In [6]:
# Para cada linha nos dados lidos
for x in range(len(reader)):
    # Captura apenas a linha em questão
    row = list(reader.iloc[x])
    # Se ainda não houver ocorrido a análise do cabeçalho (tamanho de headers == 0)
    if len(headers) == 0:
        # headers = linha atual
        headers = reader.columns
        # para todas as colunas na linha, o tamanho recebe 0 e o tipo recebe ''
        for col in headers:
            longest.append(0)
            type_list.append('')
    # Para cada coluna na linha
    for i in range(len(row)):
        # Caso já tenha sido definido como varchar, timestamp ou possua valor NA, nada é feito
        # NA é referente ao valor null no csv
        if type_list[i] == 'varchar' or str(row[i]) == 'NA' or type_list[i] == 'timestamp':
            pass
        # Caso contrário
        else:
            # Chama função para determinar tipos das colunas
            type_list[i] = dataType(str(row[i]), type_list[i])
        # Atribui tamanhos para as variáveis
        if len(str(row[i])) > longest[i]:
            longest[i] = len(str(row[i]))

# Geração do script sql para criação de tabela

In [7]:
# Cria variável para armazenar o codigo de criação da tabela
statement = 'create table ' + table_name + '('

# Escreve o nome e tipo variável por variável no código
for i in range(len(headers)):
    if type_list[i] == 'varchar':
        statement = (statement + '\n\t{} varchar({}),').format(headers[i].lower(), str(longest[i]))
    else:
        statement = (statement + '\n\t' + '{} {}' + ',').format(headers[i].lower(), type_list[i])
# Finaliza código com fechamento de parêntesis e ponto e vírgula
statement = statement[:-1] + '\n);'
# Imprime o código gerado
print(statement)

create table XPTO(
	nf varchar(2),
	id_produto varchar(2),
	qtd smallint,
	valor decimal,
	desconto_percentual smallint,
	data timestamp,
	id_cliente varchar(2)
);
